In [1]:
import sys
import time
import numpy as np
import pickle
import importlib

In [2]:
from qiskit import Aer, IBMQ
from qiskit.ignis.mitigation.measurement import TensoredMeasFitter

In [3]:
max_size = 27
max_length = 26

In [4]:
with open("pkls/raw_hist_list.pkl", "rb") as f:
    raw_hist_list = pickle.load(f)
with open("pkls/results_meas_cal.pkl", "rb") as f:
    results_meas_cal = pickle.load(f)

In [5]:
import libs_qrem
importlib.reload(libs_qrem)
from libs_qrem import LeastNormFilter

In [6]:
xz_lnp_mitigator_list = []
for n in range(2, max_size + 1): # [2, ..., 57]
    mit_pattern = [[i] for i in range(n)]
    meas_fitter = TensoredMeasFitter(results_meas_cal[n - 2], mit_pattern=mit_pattern)
    xz_lnp_mitigator_list.append(LeastNormFilter(n, meas_fitter.cal_matrices))
    if n % 10 == 0:
        print("size", n, "finished")
print("length of xz_lnp_mitigator_list: ", len(xz_lnp_mitigator_list))

size 10 finished
size 20 finished
length of xz_lnp_mitigator_list:  26


In [7]:
zx_lnp_mitigator_list = []
for n in range(2, max_size + 1): # [2, ..., 57]
    mit_pattern = [[i] for i in range(n)]
    meas_fitter = TensoredMeasFitter(results_meas_cal[n - 2], mit_pattern=mit_pattern)
    zx_lnp_mitigator_list.append(LeastNormFilter(n, meas_fitter.cal_matrices))
    if n % 10 == 0:
        print("size", n, "finished")
print("length of zx_lnp_mitigator_list: ", len(zx_lnp_mitigator_list))

size 10 finished
size 20 finished
length of zx_lnp_mitigator_list:  26


In [8]:
lnp_hist_list = []
xz_lnp_total_times = []
zx_lnp_total_times = []
for i in range(max_length):
    print(i + 2, "qubits")
    
    t1 = time.perf_counter()
    lnp_hist_list.append( xz_lnp_mitigator_list[i].apply(raw_hist_list[2 * i]) )
    t2 = time.perf_counter()
    xz_lnp_total_times.append(t2 - t1)
    print("xz", t2 - t1, "s")
    
    t3 = time.perf_counter()
    lnp_hist_list.append( zx_lnp_mitigator_list[i].apply(raw_hist_list[2 * i + 1]) )
    t4 = time.perf_counter()
    zx_lnp_total_times.append(t4 - t3)
    print("zx", t4 - t3, "s")
    
    print()

2 qubits
xz 7.790199999924141e-05 s
zx 3.124699999990099e-05 s

3 qubits
xz 4.869300000009957e-05 s
zx 4.050700000046703e-05 s

4 qubits
xz 8.996100000047136e-05 s
zx 7.885700000009876e-05 s

5 qubits
xz 0.00014157500000067103 s
zx 0.0001341129999996582 s

6 qubits
xz 0.0002667299999998818 s
zx 0.00020241100000006895 s

7 qubits
xz 0.0007970310000002812 s
zx 0.00043953600000001813 s

8 qubits
xz 0.0016313139999999393 s
zx 0.0006335479999997062 s

9 qubits
xz 0.005540777000000219 s
zx 0.0007751639999993287 s

10 qubits
xz 0.02002109499999971 s
zx 0.001421848999999753 s

11 qubits
xz 0.07620078099999983 s
zx 0.001637008999999523 s

12 qubits
xz 0.23211104700000007 s
zx 0.0025936939999997577 s

13 qubits
xz 0.543449464 s
zx 0.0048544969999992915 s

14 qubits
xz 0.8915726590000004 s
zx 0.00686340100000038 s

15 qubits
xz 1.125659443 s
zx 0.009669348000000966 s

16 qubits
xz 1.3396600749999994 s
zx 0.011428976999999563 s

17 qubits
xz 1.5397049359999997 s
zx 0.018971488000000036 s

18 qubit

In [9]:
with open("./pkls/lnp_hist_list.pkl", "wb") as f:
    pickle.dump(lnp_hist_list, f)

In [10]:
with open("./pkls/lnp_total_times.pkl", "wb") as f:
    pickle.dump({"xz": xz_lnp_total_times, "zx": zx_lnp_total_times}, f)

In [11]:
xz_lnp_times = []
zx_lnp_times = []
for i in range(max_length):
    xz_lnp_times.append( xz_lnp_mitigator_list[i].times() )
    zx_lnp_times.append( zx_lnp_mitigator_list[i].times() )

In [12]:
with open("./pkls/lnp_times.pkl", "wb") as f:
    pickle.dump({"xz": xz_lnp_times, "zx": zx_lnp_times}, f)

In [13]:
xz_lnp_mitigators = []
for i in range(max_length):
    xz_lnp_mitigators.append({"indices_to_keys_vector":xz_lnp_mitigator_list[i].indices_to_keys_vector(),
                              # "mitigated_hist":xz_lnp_mitigator_list[i].mitigated_hist(),
                              "exact_one_norm_of_reduced_inv_A":xz_lnp_mitigator_list[i].exact_one_norm_of_reduced_inv_A(),
                              # "reduced_inv_A":xz_lnp_mitigator_list[i].reduced_inv_A(),
                              "sum_of_x":xz_lnp_mitigator_list[i].sum_of_x(),
                              "sum_of_x_hat":xz_lnp_mitigator_list[i].sum_of_x_hat(),
                              "sum_of_x_tilde":xz_lnp_mitigator_list[i].sum_of_x_tilde(),
                              # "times":xz_lnp_mitigator_list[i].times(),
                              "x_s":xz_lnp_mitigator_list[i].x_s(),
                              "x_hat":xz_lnp_mitigator_list[i].x_hat(),
                              "x_tilde":xz_lnp_mitigator_list[i].x_tilde(),
                              })
    print(i, "finished")

0 finished
1 finished
2 finished
3 finished
4 finished
5 finished
6 finished
7 finished
8 finished
9 finished
10 finished
11 finished
12 finished
13 finished
14 finished
15 finished
16 finished
17 finished
18 finished
19 finished
20 finished
21 finished
22 finished
23 finished
24 finished
25 finished


In [14]:
zx_lnp_mitigators = []
for i in range(max_length):
    zx_lnp_mitigators.append({"indices_to_keys_vector":zx_lnp_mitigator_list[i].indices_to_keys_vector(),
                              # "mitigated_hist":zx_lnp_mitigator_list[i].mitigated_hist(),
                              "exact_one_norm_of_reduced_inv_A":zx_lnp_mitigator_list[i].exact_one_norm_of_reduced_inv_A(),
                              # "reduced_inv_A":zx_lnp_mitigator_list[i].reduced_inv_A(),
                              "sum_of_x":zx_lnp_mitigator_list[i].sum_of_x(),
                              "sum_of_x_hat":zx_lnp_mitigator_list[i].sum_of_x_hat(),
                              "sum_of_x_tilde":zx_lnp_mitigator_list[i].sum_of_x_tilde(),
                              # "times":zx_lnp_mitigator_list[i].times(),
                              "x_s":zx_lnp_mitigator_list[i].x_s(),
                              "x_hat":zx_lnp_mitigator_list[i].x_hat(),
                              "x_tilde":zx_lnp_mitigator_list[i].x_tilde(),
                              })
    print(i, "finished")

0 finished
1 finished
2 finished
3 finished
4 finished
5 finished
6 finished
7 finished
8 finished
9 finished
10 finished
11 finished
12 finished
13 finished
14 finished
15 finished
16 finished
17 finished
18 finished
19 finished
20 finished
21 finished
22 finished
23 finished
24 finished
25 finished


In [15]:
with open("./pkls/lnp_mitigator_list.pkl", "wb") as f:
    pickle.dump({"xz": xz_lnp_mitigators, "zx": zx_lnp_mitigators}, f)